# Scraping list of articles and dates

In [1]:
import numpy as np
import pandas as pd
import requests
import re
import sqlite3
import time

In [2]:
db = sqlite3.connect("scraping.db")

In [3]:
def create_article_table(db):
    db.execute("drop table if exists article")

    db.commit()
    db.execute("""
    create table if not exists article (id integer primary key, headline varchar(255), time timestamp, content text);
    """);

    db.commit()

In [4]:
create_article_table(db)

In [4]:
def insert_article(id, title=None, timestamp=None, content=None, feedback=None, commit=True):
    content = content.replace('"', "'") if content else None
    title = title.replace('"', "'") if title else None
    try:
        db.execute(f'insert into article values ({id}, "{title}", "{timestamp}", "{content}", "{feedback}")');
    except sqlite3.IntegrityError as e:
        print(id, " was a duplicate")
    db.commit() if commit else None

In [7]:
cur = db.execute("select * from article")
db.commit()
for row in cur:
    print(row)
    break

(848394, 'None', '2016-05-24 20:36:29', 'None')


In [21]:
url = "https://armenpress.am/eng/ajax/"
method = "POST"

def get_article_list(id):
    form_data = {
        "xdo": "news",
        "lastid": id
    }
    
    resp = requests.post(url, form_data)
    (article_ids := set(re.findall('<a href="/eng/news/(\d+).html"', resp.text)))
    timestamps = [i.replace("T", " ").split("+")[0] for i in re.findall('itemprop="datePublished"\s+content="(.+)"', resp.text)]
    return [int(i) for i in article_ids], timestamps


In [22]:
lastid = 3279

while True:
    article_ids, timestamps = get_article_list(lastid)

    if len(article_ids) == 0:
        break
    lastid += 1

    [insert_article(article_ids[i], None, timestamps[i], None, None, False) for i in range(len(article_ids))]
    db.commit()
    # time.sleep(0.15)
    print(lastid, end="\r")

514564  was a duplicate


In [16]:
cur = db.execute("select count(*) from article")
for row in cur:
    print(row)

(129200,)


In [15]:
db.commit()